In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join queueing network
val model = Network("model")

In [ ]:
// Create network 
val source = Source(model, "Source")

val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)

val fork = Fork(model, "Fork")
fork.setTasksPerLink(2)
val join = Join(model, "Join", fork)

val sink = Sink(model, "Sink")

In [ ]:
// Create job classes
val jobclass1 = OpenClass(model, "class1")
val jobclass2 = OpenClass(model, "class2")

In [ ]:
// Set arrival and service processes
// Class 1
source.setArrival(jobclass1, Exp(0.25))
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(0.75))
// Class 2
source.setArrival(jobclass2, Exp(0.25))
queue1.setService(jobclass2, Immediate())
queue2.setService(jobclass2, Exp(2.0))

In [ ]:
// Set routing matrix
val P = model.initRoutingMatrix()
// Class 1 routing
P.set(jobclass1, jobclass1, source, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, sink, 1.0)
// Class 2 routing
P.set(jobclass2, jobclass2, source, fork, 1.0)
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, sink, 1.0)
model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))
solvers.add(MVA(model))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This multi-class fork-join with task multiplication demonstrates:

1. **Task Multiplication**:
   - Fork creates 2 tasks per link (setTasksPerLink(2))
   - Each arriving job generates 4 parallel tasks total
   - 2 tasks go to Queue1, 2 tasks go to Queue2

2. **Class Differentiation**:
   - **Class 1**: Normal processing (Queue1: rate 1.0, Queue2: rate 0.75)
   - **Class 2**: Fast Queue1 (Immediate), slower Queue2 (rate 2.0)
   - Arrival rate 0.25 for both classes

3. **Service Characteristics**:
   - **Queue1**: Class1 (1.0), Class2 (Immediate - no delay)
   - **Queue2**: Class1 (0.75), Class2 (2.0)
   - Asymmetric performance between queues and classes

4. **Load Amplification**:
   - **Arrival rate**: 0.25 per class = 0.5 total
   - **Effective load**: 0.5 × 2 tasks per link = 1.0 load per queue
   - Task multiplication creates high utilization

5. **Performance Bottlenecks**:
   - **Class 1**: Queue2 (rate 0.75) likely bottleneck due to lower rate
   - **Class 2**: Queue2 still processes at rate 2.0 but gets high load
   - **Overall**: Queue2 experiences compound load from both classes

6. **Synchronization Complexity**:
   - Join waits for 2 tasks from Queue1 AND 2 tasks from Queue2
   - Must synchronize 4 parallel tasks per original job
   - Response time includes waiting for slowest of 4 tasks

The model represents:
- **Parallel computing**: Jobs that spawn multiple threads/processes
- **Manufacturing**: Parts that require multiple parallel operations
- **Data processing**: Records that generate multiple parallel tasks
- **Service systems**: Requests that trigger multiple parallel activities

Key insights:
- **Load multiplication**: Task multiplication can quickly saturate resources
- **Class interaction**: Different service rates create complex competition patterns
- **Synchronization overhead**: Waiting for multiple tasks increases response time

**Performance Pattern**: 
- Class 2's immediate service at Queue1 means its bottleneck is entirely at Queue2
- Class 1 experiences delays at both queues, with Queue2 being more constraining
- The task multiplication (2 per link) means each job requires significant parallel resources

**Design Consideration**: Task multiplication should be used carefully as it can lead to resource saturation even with moderate arrival rates.